In [1]:
%load_ext autoreload
%autoreload 2
import manenv
from manenv.utils.product_utils import *
from manenv.core.world import World
from manenv.core.component import *
from manenv.render_wrapper import RenderWrapper
from manenv.core.product import *
from manenv.core.effector import *

import manenv.effectors as eff
import manenv.components as cmp


pygame-ce 2.4.1 (SDL 2.28.5, Python 3.11.9)


In [2]:
world = World((3, 3))
s = np.zeros((5, 5), dtype=np.int8)
s[1][2] = 2
s[1][1] = 2
s[2][1] = 1
s[3][1] = 1
product = Product(s)
world.place_component(make_vector(1, 1), cmp.Conveyor(
    np.array([
    [0, 0, 0],
    [1, 0, -1],
    [0, 0, 0]
])))
world.place_component(make_vector(2, 0), cmp.Conveyor(
    np.array([
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, 0] 
])))
world.place_component(make_vector(2, 1), cmp.Outport())

grabber = eff.Grabber(make_vector(0, 2))
buddy = eff.Grabber(make_vector(1, 3))
welder = eff.Welder(make_vector(1, 2))
transformer = eff.Transformer(1, 2, make_vector(2,  4))
discard = eff.Acceptor(make_vector(1, 7))

world.place_component(make_vector(1, 2), cmp.Spawner(product))
assembler : cmp.Assembler = cmp.Assembler(make_vector(10, 10), [grabber, buddy, welder, discard, transformer])
world.place_component(make_vector(1, 0), assembler)
wrapper = RenderWrapper(world)

In [3]:
wrapper.reset()
assembler.place_in_inventory(product.copy())
assembler.place_in_inventory(product.copy())
world.update()

grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
buddy.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
world.update()

grabber.set_action(eff.GrabberActions.RELEASE.value)
buddy.set_action(eff.GrabberActions.RELEASE.value)
world.update()

welder.set_action(eff.WelderActions.WELD_EAST.value)
world.update()
world.update()

grabber.set_action(eff.GrabberActions.GRAB.value)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
world.update()

grabber.set_action(eff.GrabberActions.RELEASE.value)
world.update()

transformer.set_action(eff.TransformerActions.TRANSFORM.value)
world.update()
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
world.update()
world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
# world.update()


None [0 2]
None [1 3]
7 [0 2]
7 [1 3]
6 [0 2]
6 [1 3]
None [0 2]
None [1 3]
None [0 2]
None [1 3]
5 [0 2]
None [1 3]
2 [0 2]
None [1 3]
6 [0 3]
None [1 3]
None [0 3]
None [1 3]
None [0 3]
None [1 3]
2 [0 3]
None [1 3]
None [0 4]
None [1 3]


In [4]:
wrapper.render()

# Gym Environment Test

In [6]:
from manenv.gym_wrapper import MARLFactoryEnvironment

In [9]:
world.reset()
environment = MARLFactoryEnvironment(world)
renderer = RenderWrapper(environment._world)

In [10]:
for _ in range(100):
    action = environment.action_space.sample()
    environment.step(action)
renderer.render()

4 [0 2]
6 [1 3]
0 [0 2]
1 [1 3]
5 [0 2]
4 [1 2]
4 [0 2]
8 [0 2]
4 [0 2]
3 [0 2]
8 [0 2]
8 [1 2]
4 [0 2]
3 [1 2]
9 [0 2]
8 [2 2]
0 [0 2]
8 [2 2]
9 [0 2]
5 [2 2]
1 [0 2]
8 [2 2]
7 [0 1]
4 [2 2]
7 [0 1]
7 [1 2]
3 [0 1]
4 [1 2]
7 [1 1]
7 [0 2]
3 [1 1]
4 [0 2]
0 [2 1]
2 [0 2]
5 [2 1]
4 [0 3]
9 [2 1]
2 [0 3]
8 [2 1]
7 [0 4]
3 [2 1]
7 [0 4]
5 [3 1]
4 [0 4]
2 [3 1]
9 [0 4]
3 [3 2]
5 [0 4]
2 [4 2]
7 [0 4]
7 [4 3]
0 [0 4]
7 [4 3]
4 [0 4]
4 [4 3]
4 [0 4]
4 [3 3]
1 [0 4]
3 [2 3]
9 [0 3]
1 [3 3]
5 [0 3]
3 [3 2]
7 [0 3]
1 [4 2]
6 [0 3]
3 [4 1]
8 [0 3]
5 [5 1]
2 [0 3]
6 [5 1]
0 [0 4]
2 [5 1]
4 [0 4]
3 [5 2]
4 [0 4]
2 [6 2]
6 [0 4]
2 [6 3]
8 [0 4]
5 [6 4]
2 [0 4]
9 [6 4]
2 [0 5]
3 [6 4]
7 [0 6]
3 [7 4]
7 [0 6]
3 [8 4]
3 [0 6]
6 [9 4]
9 [1 6]
6 [9 4]
4 [1 6]
8 [9 4]
1 [0 6]
3 [9 4]
8 [0 5]
2 [9 4]
9 [0 5]
1 [9 5]
3 [0 5]
0 [9 4]
9 [1 5]
8 [9 4]
0 [1 5]
8 [9 4]
3 [1 5]
4 [9 4]
1 [2 5]
3 [8 4]
5 [2 4]
0 [9 4]
7 [2 4]
2 [9 4]
8 [2 4]
9 [9 5]
7 [2 4]
8 [9 5]
8 [2 4]
7 [9 5]
3 [2 4]
5 [9 5]
4 [3 4]
5 [9 5]
